In [1]:
import warnings
warnings.filterwarnings('ignore')

### Import the sql dataset

In [2]:
import sqlite3
import pandas as pd

connection = sqlite3.connect('database.sqlite')

# polarisable_dataset = dataset that contains Score = {1,2,4,5} assuming Score = 3 implies neutral comments and
# Score < 3 implies negative comment and Score > 3 implies positive comment
polarisable_dataset = pd.read_sql_query('select * from REVIEWS WHERE Score != 3', connection)
polarisable_dataset.shape

(525814, 10)

### Replace values in Score column in polarisable dataset with 'positive' and 'negative'

In [3]:
scores = polarisable_dataset['Score']

polarised_scores = scores.map(lambda x: 0 if x<3 else 1)

# polarised_scores.head()

polarisable_dataset['Score'] = polarised_scores
polarised_dataset = polarisable_dataset

In [4]:
polarised_dataset.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,1,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,0,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,1,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,0,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,1,1350777600,Great taffy,Great taffy at a great price. There was a wid...


### Exploratory Data Analysis

##### 1. Deduplication
If a user id has multiple entries for the same timestamp, then it should be removed because it is likely that multiple entries at the same timestamp were for the same product of different variety which has a different product id than other variants

##### Observation 1- There are 197082 duplicate entries (using rule-1)

In [5]:
print(polarised_dataset.duplicated(['UserId', 'Time']).sum())



197082


In [6]:
deduplicated_dataset = polarised_dataset.drop_duplicates(subset = {'UserId', 'Time'}, keep = 'first', inplace = False)
deduplicated_dataset.shape

(328732, 10)

#### 2. Text preprocessing- Removing html and punctuations

In [7]:
deduplicated_dataset.columns

Index(['Id', 'ProductId', 'UserId', 'ProfileName', 'HelpfulnessNumerator',
       'HelpfulnessDenominator', 'Score', 'Time', 'Summary', 'Text'],
      dtype='object')

In [8]:
deduplicated_dataset['Text'].head(1)

0    I have bought several of the Vitality canned d...
Name: Text, dtype: object

In [9]:
# dataset cleaners

import re

def remove_html(sentence):
    html_tag_re_obj = re.compile('<.*>?')
    return re.sub(html_tag_re_obj, ' ', sentence)

def remove_punctuations(sentence):
    cleaned_sentence = re.sub(r'[^a-zA-Z]', r' ', sentence)
    return cleaned_sentence

In [10]:
# clean dataset

corpus = deduplicated_dataset['Text']

cleaned_corpus = []
for doc in corpus.values:
    cleaned_doc = remove_html(doc)
    cleaned_doc = remove_punctuations(cleaned_doc)
    cleaned_corpus.append(cleaned_doc)

deduplicated_dataset['Text'] = cleaned_corpus

#### 2. Text preprocessing- Removing stop words

In [11]:
from nltk.corpus import stopwords

In [12]:
print(set(stopwords.words('english')))

{'yourself', 'yours', 'should', 'but', 'by', 'be', 'down', 'hasn', "she's", 'has', 'with', 'no', 'can', 'now', 'any', "hadn't", 'wouldn', 'theirs', 'at', 'itself', 'most', 't', 'did', "weren't", "doesn't", 'we', 'some', "isn't", 'yourselves', 'mustn', 'our', 'very', 'her', 'for', 'doesn', 'on', 'then', 'have', 've', 'couldn', 'ma', 'about', 'him', 'so', 'their', 'off', 'she', 'those', 'having', 'if', 'its', 'when', 'do', "couldn't", 'was', 'won', 'before', 'out', 'being', 'y', 'ourselves', 'an', 'the', 'mightn', 'does', 'such', 'other', 'your', 're', 'or', 'shan', 'why', 'his', 'had', 'than', 'ain', 'through', 'am', "it's", 'are', 'same', 'whom', 'haven', 'from', 'over', 'until', "wasn't", 'herself', 'my', 'what', 'don', 'themselves', 'in', 'aren', 'between', 'you', 'it', 'where', "mustn't", 'again', "won't", 'that', 'each', 'after', 'doing', "haven't", 'not', 'll', "you've", 'is', "shouldn't", 'more', 'me', "should've", "shan't", 'o', 'further', 'shouldn', 'he', 'hadn', "hasn't", 'wer

In [13]:
## Since the negative food reviews are likely to contain words like "don't", "didn't", etc that impart important
## meaning to the review, we check if such words exist in the corpus that we have. If these words are in the corpus,
## then they should not be in the list of stop words that we use for removing the stopwords from our corpus

count = 0
for doc in deduplicated_dataset['Text']:
    if "not" in doc:
        count += 1

print(count)

count = 0
for doc in deduplicated_dataset['Text']:
    if "don't" in doc:
        count += 1

print(count)

count = 0
for doc in deduplicated_dataset['Text']:
    if "didn't" in doc:
        count += 1

print(count)

104301
0
0


In [14]:
stopwords = stopwords.words('english')

In [15]:
## 'not' is present in 104301 docs in the corpus, so we modify the list of stopwords to not contain this word

stopwords_set = set(stopwords)

stopwords_set.remove('not')

In [16]:
print(stopwords_set)

{'yourself', 'yours', 'should', 'but', 'by', 'be', 'down', 'hasn', "she's", 'has', 'with', 'no', 'can', 'now', 'any', "hadn't", 'wouldn', 'theirs', 'at', 'itself', 'most', 't', 'did', "weren't", "doesn't", 'we', 'some', "isn't", 'yourselves', 'mustn', 'our', 'very', 'her', 'for', 'doesn', 'on', 'then', 'have', 've', 'couldn', 'ma', 'about', 'him', 'so', 'their', 'off', 'she', 'those', 'having', 'if', 'its', 'when', 'do', "couldn't", 'was', 'won', 'before', 'out', 'being', 'y', 'ourselves', 'an', 'the', 'mightn', 'does', 'such', 'other', 'your', 're', 'or', 'shan', 'why', 'his', 'had', 'than', 'ain', 'through', 'am', "it's", 'are', 'same', 'whom', 'haven', 'from', 'over', 'until', "wasn't", 'herself', 'my', 'what', 'don', 'themselves', 'in', 'aren', 'between', 'you', 'it', 'where', "mustn't", 'again', "won't", 'that', 'each', 'after', 'doing', "haven't", 'll', "you've", 'is', "shouldn't", 'more', 'me', "should've", "shan't", 'o', 'further', 'shouldn', 'he', 'hadn', "hasn't", 'were', 'my

In [49]:
## lower casing all docs in corpus (deduplicated_dataset['Text'])

lower_cased_docs = [doc.lower() for doc in deduplicated_dataset['Text']]
deduplicated_dataset['Text'] = lower_cased_docs

In [50]:
corpus = deduplicated_dataset['Text'] # corpus contains cleaned docs
print(type(corpus))

<class 'pandas.core.series.Series'>


In [51]:
docs_without_stop_words = []
for i, doc in enumerate(corpus):
    non_stop_words_in_doc = []
    for word in doc.split():
        if word not in stopwords_set:
            non_stop_words_in_doc.append(word)
            
    
    docs_without_stop_words.append(' '.join(non_stop_words_in_doc))

In [52]:
print(docs_without_stop_words[:10])

['bought several vitality canned dog food products found good quality product looks like stew processed meat smells better labrador finicky appreciates product better', 'product arrived labeled jumbo salted peanuts peanuts actually small sized unsalted not sure error vendor intended represent product jumbo', 'confection around centuries light pillowy citrus gelatin nuts case filberts cut tiny squares liberally coated powdered sugar tiny mouthful heaven not chewy flavorful highly recommend yummy treat familiar story c lewis lion witch wardrobe treat seduces edmund selling brother sisters witch', 'looking secret ingredient robitussin believe found got addition root beer extract ordered good made cherry soda flavor medicinal', 'great taffy great price wide assortment yummy taffy delivery quick taffy lover deal', 'got wild hair taffy ordered five pound bag taffy enjoyable many flavors watermelon root beer melon peppermint grape etc complaint bit much red black licorice flavored pieces not 

#### 2. Text preprocessing- Stemming

In [44]:
from nltk.stem import SnowballStemmer

In [46]:
stemmer = SnowballStemmer('english')

In [57]:
stemmer.stem('found')

'found'

In [53]:
stemmed_corpus = [] # docs with stemmed words
for doc in docs_without_stop_words:
    stemmed_words = []
    for word in doc.split():
        stemmed_words.append(stemmer.stem(word))
    stemmed_doc = ' '.join(stemmed_words)
    stemmed_corpus.append(stemmed_doc)

In [54]:
print(stemmed_corpus[:10])

['bought sever vital can dog food product found good qualiti product look like stew process meat smell better labrador finicki appreci product better', 'product arriv label jumbo salt peanut peanut actual small size unsalt not sure error vendor intend repres product jumbo', 'confect around centuri light pillowi citrus gelatin nut case filbert cut tini squar liber coat powder sugar tini mouth heaven not chewi flavor high recommend yummi treat familiar stori c lewi lion witch wardrob treat seduc edmund sell brother sister witch', 'look secret ingredi robitussin believ found got addit root beer extract order good made cherri soda flavor medicin', 'great taffi great price wide assort yummi taffi deliveri quick taffi lover deal', 'got wild hair taffi order five pound bag taffi enjoy mani flavor watermelon root beer melon peppermint grape etc complaint bit much red black licoric flavor piec not particular favorit kid husband last two week would recommend brand taffi delight treat', 'saltwat 